### Clustering people based on appearance (re-id)

1. Get a dataset with full frames and crop people and save then.
2. Extract embeddings for each people appearence.
3. Create a visuzalization in fiftyone so I can cluster and annotate easily.

In [ ]:
import os, json, random
from PIL import Image
from tqdm import tqdm

DATASET_DIR = "/home/gfuhr/projects/data/immediate_action/people_v1/"
OUTPUT_DIR = "/home/gfuhr/projects/data/immediate_action/people_v1_cropped/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

dataset_annotations = os.path.join(DATASET_DIR, "annotations.json")
with open(dataset_annotations, "r") as f:
    ann = json.load(f)

images = {img["id"]: img for img in ann["images"]}
anns = ann["annotations"]

for a in tqdm(anns):
    img_info = images[a["image_id"]]
    img_path = os.path.join(DATASET_DIR, img_info["file_name"])
    if not os.path.exists(img_path): continue
    x, y, w, h = map(int, a["bbox"])
    im = Image.open(img_path)
    crop = im.crop((x, y, x + w, y + h))
    out_path = os.path.join(OUTPUT_DIR, f"{a['id']}.png")
    crop.save(out_path)


In [ ]:
import fiftyone as fo
import glob

dataset_name = "people_v1_cropped"
if dataset_name in fo.list_datasets():
    fo.delete_dataset(dataset_name)
dataset = fo.Dataset(dataset_name)
image_paths = sorted(glob.glob(os.path.join(OUTPUT_DIR, "*.png")))

samples = [fo.Sample(filepath=img_path) for img_path in image_paths]
dataset.add_samples(samples)


In [ ]:
import os
import numpy as np
import fiftyone.brain as fob
import fiftyone as fo

dataset_embeddings = np.load("reid/reid_features_sbs_r101.npz")

dataset = fo.load_dataset("people_v1_cropped")
embeddings = []

for sample in dataset:
    filename = os.path.basename(sample.filepath)
    if filename in dataset_embeddings:
        # print(dataset_embeddings[filename].shape)
        # break
        embeddings.append(dataset_embeddings[filename])
    else:
        print(f"Warning: {filename} not found in dataset embeddings")
        break
        
    # else: 

feature_matrix = np.vstack(embeddings)  # shape: (n_images, 512)
print(feature_matrix.shape)


results = fob.compute_visualization(
    dataset, embeddings=feature_matrix, seed=51, brain_key="reid_embeddings_sbs_r101"
)